In [ ]:
import socket
import threading
import os

server = socket.socket()
server.bind(("localhost", 9996))
server.listen(10)

print(">> Secure chat server started... waiting for users")

active_clients = []

ALLOWED_EXTENSIONS = [".txt", ".jpg", ".pdf"]
BANNED_WORDS = ["spam", "hack", "sensitive"]

# HANDLE CLIENT THREAD 
def handle_client(conn, addr):
    print(">> User connected:", addr)
    active_clients.append(conn)

    while True:
        try:
            data = conn.recv(1024).decode()

            if not data:
                continue

            # MESSAGE HANDLING 
            if data.startswith("MSG:"):
                msg = data[4:]

                if msg.lower() == "exit":
                    print(">> User left:", addr)
                    active_clients.remove(conn)
                    conn.close()
                    break

                if any(word in msg.lower() for word in BANNED_WORDS):
                    conn.send("Message blocked: inappropriate content".encode())
                else:
                    print(f"{addr} says:", msg)
                    for client in active_clients:
                        if client != conn:
                            client.send(f"{addr} says: {msg}".encode())

            # FILE HANDLING 
            elif data.startswith("FILE:"):
                filename = data[5:]
                ext = os.path.splitext(filename)[1]

                if ext not in ALLOWED_EXTENSIONS:
                    conn.send("File rejected: type not allowed".encode())
                    continue

                # Receive file size first
                import struct
                filesize_bytes = conn.recv(4)
                filesize = struct.unpack("!I", filesize_bytes)[0]

                # Receive file content
                received = 0
                file_data = b""
                while received < filesize:
                    chunk = conn.recv(min(4096, filesize - received))
                    if not chunk:
                        break
                    file_data += chunk
                    received += len(chunk)

                # Save file
                with open(filename, "wb") as f:
                    f.write(file_data)

                print(f">> {addr} sent file: {filename}")

                for client in active_clients:
                    if client != conn:
                        client.send(f"File received from {addr}: {filename}".encode())

        except:
            if conn in active_clients:
                active_clients.remove(conn)
            conn.close()
            break

# ACCEPT CLIENTS 
while True:
    conn, addr = server.accept()
    threading.Thread(target=handle_client, args=(conn, addr)).start()


>> Secure chat server started... waiting for users
>> User connected: ('127.0.0.1', 57101)
>> User connected: ('127.0.0.1', 53805)
>> User connected: ('127.0.0.1', 50898)
('127.0.0.1', 50898) says: hi everyone
('127.0.0.1', 53805) says: send me the file
>> ('127.0.0.1', 57101) sent file: file1.txt
('127.0.0.1', 57101) says: thanku
